In [4]:
# Active Learning - Uncertainty Sampling

import numpy as np
import pandas as pd
import os
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import RandomForestClassifier
import joblib
from sklearn.calibration import CalibratedClassifierCV
from warnings import simplefilter
from sklearn.metrics import confusion_matrix, classification_report, f1_score

from google.colab import drive


simplefilter(action='ignore', category=FutureWarning)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported fro

In [5]:
# Συνδέομαι στο google drive για να μπορώ να αποθηκεύω και να ανακτώ μοντέλα από εκεί
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
# Καθορίζω την διαδρομή στο google drive και φορτώνω στην μνήμη το σύνολο των δεδομένων
path = 'drive/My Drive/bank_credit'
dt = pd.read_csv(os.path.join(path, 'creditcard.csv'))

In [7]:
# Επισκόπηση λίγων δεδομένων των δύο κλάσεων
a = dt.loc[dt['Class']==0].sample(3)
b = dt.loc[dt['Class']==1].sample(5)
c = pd.concat([a,b])
c

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
88243,62055.0,-0.823670,0.975735,0.798634,0.212036,-0.112791,-1.377781,0.696054,0.131562,-0.937579,-0.690288,-0.287602,0.452328,0.523680,0.582003,0.395690,-0.118946,-0.031702,-0.411845,-0.096015,-0.010485,0.180385,0.285744,0.026473,0.755666,-0.319416,0.231031,-0.034522,0.055759,39.15,0
240818,150762.0,2.008927,-2.263022,-1.444073,-2.452135,0.329183,4.081136,-2.309242,1.074643,-0.733777,1.438138,-0.331336,-0.617749,0.519785,-0.721150,0.501517,-0.166552,0.238824,0.160813,-0.491428,-0.123679,-0.080723,-0.025031,0.292082,0.678721,-0.548289,-0.236406,0.066188,-0.017307,122.00,0
240459,150610.0,-1.295840,1.755734,0.051123,1.143860,-0.710480,0.411713,-0.540472,1.375232,-0.149230,-0.931242,-1.924674,0.583753,-0.270315,0.635943,-1.002578,-0.549747,0.790798,-0.489030,1.179576,-0.475310,-0.271376,-0.988861,0.343485,0.593283,-0.765113,-1.051125,-0.361023,0.017319,27.76,0
92777,64093.0,-6.133987,2.941499,-5.593986,3.258845,-5.315512,-0.637328,-4.476488,1.695994,-1.606743,-5.117259,3.977222,-8.066096,-0.760888,-8.660771,2.220238,-6.237597,-11.046382,-4.155879,1.763611,-0.815086,0.868340,0.793736,0.217347,-0.021985,0.145882,0.665088,-1.684186,0.310195,294.90,1
149869,92092.0,-1.108478,3.448953,-6.216972,3.021052,-0.529901,-2.551375,-2.001743,1.092432,-0.836098,-4.095649,2.425951,-6.067262,1.832433,-3.501703,-1.004008,-3.501281,-2.941876,-1.125741,0.373715,-0.068598,0.825951,1.144170,0.208559,-0.295497,-0.690232,-0.364749,0.229327,0.208830,18.00,1
39183,39729.0,-0.964567,-1.643541,-0.187727,1.158253,-2.458336,0.852222,2.785163,-0.303609,0.940006,-1.965309,0.159744,-0.490697,-1.181977,-1.958876,1.152743,-1.341269,2.498325,0.777831,1.406045,1.784449,0.447180,0.536204,1.634061,0.203839,0.218749,-0.221886,-0.308555,-0.164500,776.83,1
140786,83934.0,-0.433222,2.428379,-3.996454,4.871299,-1.796308,-0.586868,-4.654543,1.285230,-2.743539,-5.638941,4.343743,-6.655734,-0.293109,-9.505141,-1.396149,-4.850323,-10.006253,-2.789598,0.982584,0.745029,0.713559,-0.408954,-0.320890,-0.804230,0.962852,0.199558,1.094533,0.541148,1.00,1
99506,67150.0,-1.824295,0.403327,-1.994122,2.756558,-3.139064,0.408185,-1.209045,1.095634,-1.447225,-3.951003,2.560700,-3.411983,-0.781670,-4.732768,-0.433974,-3.306088,-4.231022,-0.750810,2.690808,1.503688,0.838760,0.341727,0.947506,-0.145493,0.049326,0.831065,0.332421,0.252713,489.71,1


In [8]:
# Δημιουργώ ένα χαρακτηριστικό πυκνότητας συναλλαγών (Density) και ένα πραγματικής ώρας (Actual Time)
# θεωρώντας ότι μπορεί να υπάρχει κάποια συσχέτιση με τις κακόβουλες κινήσεις
# Ενώ παράλληλα διαγράφω το χαρακτηριστικό Time
period = 30 * 60 # secs
half_period = int(period / 2)
a = np.histogram(dt['Time'], bins=np.linspace(0, 172800, 172801))[0]

density48h = [np.mean(a[[j % 172800 for j in range(-half_period, +half_period)]])]
for i in range(1, len(a)):
  density48h.append(density48h[-1] + (a[(i + half_period - 1) % 172800] - a[(i - half_period) % 172800]) / period)

density48h = np.array(density48h)
density48h = density48h / density48h.max()

dt['Density'] = density48h[dt['Time'].astype(int).values]
dt['Density'].reset_index(drop=True)
dt['Actual_Time'] = dt['Time'].apply(lambda x: np.mod(x, 86400) / 86400)

dt.drop('Time', inplace=True, axis='columns')

In [9]:
# Μετατρέπω το χαρακτηριστικό Amount με λογαριθμική κανονικοποίηση
amounts = dt['Amount'].copy()
dt['Amount'] = np.log(dt['Amount']+2)

In [10]:
# Χωρίζω το σύνολο δεδομένων σε εκπαίδευσης, επικύρωσης και ελέγχου
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
seed = 0

x_tr, x_te, y_tr, y_te = train_test_split(dt.loc[:, dt.columns != 'Class'], dt['Class'], test_size=.3, random_state=seed)

va_size = int(len(x_tr) * .2)
x_tr, x_UN = x_tr[:va_size], x_tr[va_size:]
y_tr, y_UN = y_tr[:va_size], y_tr[va_size:]

print(x_tr.shape, x_UN.shape, x_te.shape)

(39872, 31) (159492, 31) (85443, 31)


In [11]:
# Δημιουργία ενός αριθμού από υποσύνολα δεδομένων τα οποία έχουν ίσο αριθμό παραδειγμάτων των δύο κλάσεων.
# Με αυτά εκπαιδεύονται μοντέλα Random Forest και η έκδσοση απόφασης γίνεται με ψηφοφορία.
# Το bagging μοντέλο εκπαιδεύεται και αποθηκεύεται, ώστε να μην χρειάζεται σε κάθε εκτέλεση του notebook

# y0 = y_tr[y_tr==0]
# x0 = x_tr[y_tr==0]

# y1 = y_tr[y_tr==1]
# x1 = x_tr[y_tr==1]

# ln = len(y1)
# number_of_subsets = 200

# print(f'iso {number_of_subsets}...')
# mod_iso = BalancedBaggingClassifier(base_estimator=RandomForestClassifier(n_estimators=200), n_estimators=number_of_subsets)
# model_iso = CalibratedClassifierCV(base_estimator=mod_iso, cv=3, method='isotonic')
# model_iso.fit(x_tr, y_tr)

# joblib.dump(model_iso, path + '/iso_bal_bag_rf_200_gini_200_active_learning.joblib')

In [12]:
# Ανάκληση του εκπαιδευμένου μοντέλου και υπολογισμός προβλέψεων στο σύνολο ελέγχου

model_iso = joblib.load(path + '/iso_bal_bag_rf_200_gini_200_active_learning.joblib')

y_preds = model_iso.predict_proba(x_te.to_numpy())

y_preds = y_preds.argmax(axis=1)

print(confusion_matrix(y_te, y_preds).T)
print()
print(classification_report(y_te, y_preds))


[[85279    33]
 [   17   114]]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85296
           1       0.87      0.78      0.82       147

    accuracy                           1.00     85443
   macro avg       0.93      0.89      0.91     85443
weighted avg       1.00      1.00      1.00     85443



In [ ]:
# Πρόβλεψη των καλιμπραρισμένων πιθανοτήτων για όλα τα μη ετικετοποιημένα δεδομένα και επιλογή των 50000 πιο αβέβαιων

# cost = 5000
# cost_per_example = 0.1

# preds = model_iso.predict_proba(x_UN.to_numpy())

# idx = np.argsort(np.abs(preds[:,0].flatten() - 0.5))[:int(cost/cost_per_example)]
 

In [ ]:
# Εκπαίδευση του μοντέλου με τα νέα επαυξημένα δεδομένα και αποθήκευσή του ώστε να μην επανεκπαιδεύεται σε κάθε νέα εκκίνηση του notebook

# x_ = np.vstack([x_UN.to_numpy()[idx], x_tr])
# y_ = y_UN.to_numpy()[idx].tolist() + y_tr.to_list()

# mod_iso_ = BalancedBaggingClassifier(base_estimator=RandomForestClassifier(n_estimators=200), n_estimators=number_of_subsets)
# model_iso_ = CalibratedClassifierCV(base_estimator=mod_iso_, cv=3, method='isotonic')
# model_iso_.fit(x_, y_)

# joblib.dump(model_iso_, path + '/iso_bal_bag_rf_200_gini_200_active_learning_uncertainty.joblib')


In [14]:
# Ανάκληση του αποθηκευμένου μοντέλου και πρόβλεψη των κατηγοριών των παραδειγμάτων ελέγχου
# Αυτή η προσπάθεια δεν δούλεψε καλά. Έριξε την ακρίβεια του μοντέλου για τα αρνητικά παραδείγματα η επέκταση των δεδομένων με μία μόνο εκτίμηση

model_iso_ = joblib.load(path + '/iso_bal_bag_rf_200_gini_200_active_learning_uncertainty.joblib')

y_preds_ = model_iso_.predict_proba(x_te.to_numpy())

y_preds_ = y_preds_.argmax(axis=1)

print(confusion_matrix(y_te, y_preds_).T)
print()
print(classification_report(y_te, y_preds_))

[[85283    44]
 [   13   103]]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85296
           1       0.89      0.70      0.78       147

    accuracy                           1.00     85443
   macro avg       0.94      0.85      0.89     85443
weighted avg       1.00      1.00      1.00     85443



In [ ]:
# Σταδιακή επέκταση των δεδομένων εκπαίδευσης σε 10 βήματα (ανά 5000 δείγματα) και αποθήκευση
# των επιμέρους εκπαιδευμένων μοντέλων για να μην επανεκπαιδεύονται σε κάθε επανεκκίνηση του notebook

# cost = 5000
# cost_per_example = 0.1


# sps = np.linspace(0, int(cost/cost_per_example), 11)
# x_rest = x_UN.to_numpy()
# y_rest = y_UN.to_numpy()
# x_ = x_tr.to_numpy()
# y_ = y_tr.to_numpy()

# for i in range(10):
#   print(f'{i+1}/10...')
#   sp = int(sps[i+1] - sps[i])

#   preds = model_iso.predict_proba(x_rest)

#   idx = np.argsort(np.abs(preds[:,0].flatten() - 0.5))[:sp]

#   x_ = np.vstack([x_rest[idx], x_])
#   y_ = np.array(y_rest[idx].tolist() + y_.tolist())

#   mod_iso_ = BalancedBaggingClassifier(base_estimator=RandomForestClassifier(n_estimators=200), n_estimators=number_of_subsets)
#   model_iso_ = CalibratedClassifierCV(base_estimator=mod_iso_, cv=3, method='isotonic')
#   model_iso_.fit(x_, y_)

#     joblib.dump(model_iso_, path + f'/iso_bal_bag_rf_200_gini_200_active_learning_uncertainty_{i}.joblib')

#   x_rest = np.delete(x_rest, idx, axis=0)
#   y_rest = np.delete(y_rest, idx, axis=0)

In [15]:
# Ανάκληση των αποθηκευμένων μοντέλων με τα σταδιακά αυξανόμενα δεδομένα και εκτύπωση στατιστικών
# Παρατηρείται μια μηδαμινή πτώση στα αρνητικά παραδείγματα της τάξης του 3.5x10^-5  και μια σημαντική
# βελτίωση στα θετικά παραδείγματα, της τάξης του 20%

for i in range(10):
  model_iso_ = joblib.load(path + f'/iso_bal_bag_rf_200_gini_200_active_learning_uncertainty_{i}.joblib')

  y_preds_ = model_iso_.predict_proba(x_te.to_numpy())

  y_preds_ = y_preds_.argmax(axis=1)

  print(f'Μετά από πρόσθεση {i+1}x5000 παραδειγμάτων...')
  print(confusion_matrix(y_te, y_preds_).T)
  print()
  print(classification_report(y_te, y_preds_))
  print(50*'-')

Μετά από πρόσθεση 1x5000 παραδειγμάτων...
[[85279    32]
 [   17   115]]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85296
           1       0.87      0.78      0.82       147

    accuracy                           1.00     85443
   macro avg       0.94      0.89      0.91     85443
weighted avg       1.00      1.00      1.00     85443

--------------------------------------------------
Μετά από πρόσθεση 2x5000 παραδειγμάτων...
[[85279    34]
 [   17   113]]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85296
           1       0.87      0.77      0.82       147

    accuracy                           1.00     85443
   macro avg       0.93      0.88      0.91     85443
weighted avg       1.00      1.00      1.00     85443

--------------------------------------------------
Μετά από πρόσθεση 3x5000 παραδειγμάτων...
[[85278    34]
 [   18   113]]

              precisio